<p></p><font size=8 face="courier" color="magenta">Visualize</font>

This notebook creates the widget inside of the "Visualize" section of the [Cytoscape app](../Cytoscape.ipynb).

# Setup Notebook

In [1]:
""" DEBUG """
%cd /root/Documents    
%run "Source Code/Setup Notebook.ipynb" #<-- Debug
%run "Source Code/Setup Source.ipynb"   #<-- Debug
# %cd sets the path to that of Cytoscape.ipynb


/root/Documents


In [554]:
JS("var py = Jupyter.notebook.kernel")

The **<font color="magenta">create</font>** object holds all the code for this notebook

## Helpers

These are <font color="blue">functions</font> and <font color="magenta">classes</font> that help the visualizer do it's thing.

# Create <font color="magenta">Visualizer</font>

Here we create the initial <font color="magenta">Visualizer</font> class. This class is used to creat and manage the networks visualization. This section only consists of the initialization <font color="magenta">Visualizer</font> and some of it's basic functions. Most of <font color="magenta">Visualizer's</font> functionality is actually implemented inside of the **Add Widgets** section

## <font color="magenta">Visualizer</font> Class

The <font color="magenta">Visualizer</font> Class is what is used to create and manage the networks visualization

In [2]:
class Graph():
    def __init__(self):
        #self.w = type("widgets",(),{})() #<-- this is where all the widgets go
        pass

##  <font color="blue">Functions</font>

###  <font color="blue">canvasHtml</font>

<font color="blue">canvasHtml</font> creates the html that the network will be displayed in

 <font color="red">made width it 50 just for debugging</font>

In [3]:
def canvasHtml(self): 
    display(HTML("""
    <style> .cy { width: 50vw; height: 30vw}</style> 
    <div id="cy" class="cy"></div>
    """))
Graph.canvasHtml = canvasHtml

# <font color="orange">Javascript</font> Helpers

In [4]:
class JS_Helper():
    pass
Graph.js = JS_Helper()

## <font color="orange">start</font>

<font color="orange">start</font> makes it possible to edit the canvas that displays the network and loads in the <font color="orange">javascript</font> helpers

In [5]:
def start(self):
    JS("""
    // This block creates the javascript object 'cy' which hold everything related to the graph
    var cy;
    require(['cytoscape'], function(cytoscape){       
        cy = cytoscape({ container: document.getElementById('cy'), layout: { name: 'cose' },
                         style: [ { selector: 'node', style: { 'overlay-color': 'gray', 'overlay-padding': '2px', 'background-color': 'gray'  } },
                                  { selector: 'edge', style: { 'overlay-color': 'gray', 'overlay-padding': '8px', 'line-color': 'gray'  } } ]
        });
        
        // Here we initialize some settings for 'cy' 
        cy.io = {}       // <-- where we store all the parameters/variables we create        
        cy.maxZoom(14)   // <-- Determine how much window zooms in and out
        cy.minZoom(.4)
        
        cy.on("select", 'node, edge' , function(){     //<-- highlight node when selected
            this.style( 'overlay-opacity', .4 );
        });
        cy.on("unselect", 'node, edge' , function(){   //<-- unhighlight when not selected
            this.style( 'overlay-opacity', 0 );
        });
    });
    

    """)
JS_Helper.start = start

## <font color="orange">addNode</font>

This functions adds new nodes to the network when the clicks and empty part of the graph.

In [6]:
def addNode(self):
    JS("""
    cy.io.addNode = function(event){
      if( event.target === cy ){                                                          // <-- If we tapped on the background
          checkit = cy.add([ {  group: "nodes", position: event.position  } ]);          //<-- add node    
      }; 
    };
    cy.on('tap', cy.io.addNode );
    """)
JS_Helper.addNode = addNode

## <font color="orange">addEdge</font>

We add a new edge when the use double clicks one nodes and then double clicks another node. <font color="orange">addEdge</font> check for the double clicking and adds a new edge if necessary.

In [7]:
def addEdge(self):
    JS("""
    cy.io.click_hist = ["1", "2", "3", "4"];
    
    cy.io.addEdge = function(event){                                    // <-- on "tapping" canvas
        cy.io.click_hist.shift();                                         //<-- Remove oldest click

        if( event.target !== cy && event.target.isNode() ){               // <-- If we tapped a Node
          cy.io.click_hist.push( event.target.id() );                   // <-- Add node id to click history

          if( cy.io.click_hist[0]==cy.io.click_hist[1] && cy.io.click_hist[2]==cy.io.click_hist[3] ){      //<-- If 2 double clicks
              cy.add([ {  data: { source: cy.io.click_hist[0], target: cy.io.click_hist[2] }  } ]);         //<-- add edge
          };

        }else{                                                 //<-- if we didn't click a node
          cy.io.click_hist.push( String(Math.random()) );   //<-- Add random string to click history (repeats unlikely)
        }; 
    };
    cy.on("click" , cy.io.addEdge );
    """)
JS_Helper.addEdge = addEdge

## <font color="orange">onSelect</font> 

<font color="orange">onSelect</font>  does something

In [8]:
def onSelect(self):
    JS("""
    """)
JS_Helper.onSelect = onSelect

## <font color="orange">editSelector</font>

<font color="orange">editSelector</font> decides what widgets are displayed inside of the editor box *("My Edit Box")*. What's displayed depends on if there are 1/many of node/edges/both.

In [534]:
def editSelector(self):
    JS("""
        var check = 0;
        var check_var;
        cy.io.select_type = "nothing"


        cy.io.test = function(event){
            cy.io.selection = cy.$(':selected')                 //<-- Get selected elements
            cy.io.last_type = cy.io.select_type

            if( cy.io.selection.length == 0  ){                 //<-- Do nothing if nothing's selected
                return
            }
            if( cy.io.selection.length == 1 ){                  //<-- If selection lenght is 1
                //py.execute("Graph.w.Edit.id.value = '"++"'")  //<- put selectors ID in box

                if( cy.io.selection.isNode() && cy.io.last_type != "1 node" ){                      //<-- If it's a node
                    cy.io.select_type = "1 node"
                    py.execute("Graph.editor_box.widg = Graph.box.show('1 node')")
                    IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box 
                }
                if( cy.io.selection.isEdge() &&  cy.io.last_type != "1 edge" ){                     //<-- If it's an edge
                    cy.io.select_type = "1 edge"
                    py.execute("Graph.editor_box.widg = Graph.box.show('1 edge')")
                    IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box
                }

            } else {                                           //<-- If Many elements
                console.log("many items")
            }

        };

        cy.on('select unselect', 'node, edge', cy.io.test );
    """)
JS_Helper.editSelector = editSelector

#  Add <font color="green">Widgets</font>


In [11]:
Graph.w = type("widget_list",(),{})()
Graph.wf = type("widget_functions",(),{})()

## <font color="green">Control Panel</font> 

**<font color="green">widget : </font>**

In [12]:
Graph.w.ControlPanel= widgets.ToggleButtons(
    options=[ 'Edit','Search','Graph Style'])
#Graph.w.ControlPanel

**<font color="blue">Function : </font>**  <font color="blue">ControlChanged</font> changes the editorBox display when the Control Panel changes.

In [13]:
def ControlChanged(b):
    #widg = Graph.w.ControlPanel.value.replace(" ", "")
    #Graph.editor_box.widg = getattr(Graph.w, widg).show
    #Graph.editor_box.execute()
    #clear_output()
    #display(Graph.w.ControlPanel)
    pass

    
Graph.w.ControlPanel.observe( ControlChanged )

How to change button style

In [ ]:
# Graph.w.ControlPanel.value = "Search"

# <font color="green">Edit Box Widgets</font> 

In [14]:
Graph.w.Edit = type("widget",(),{})()

## Delete 

In [432]:
Graph.w.Edit.delete = ipw.Button(description='Delete',button_style='danger')
#Graph.w.Edit.delete

In [420]:
def delete(b):
    JS("cy.io.last_deleted = cy.$(':selected').remove();")   #<-- delete elements
    Graph.w.Edit.undo.disabled = False
Graph.wf.delete = delete
Graph.w.Edit.delete.on_click( Graph.wf.delete )

## Undo Delete 

In [423]:
Graph.w.Edit.undo = ipw.Button(description='Undo',button_style='primary', disabled= True )
#Graph.w.Edit.undo

In [422]:
def undo(b):
    JS(" cy.add( cy.io.last_deleted );")   #<-- Add last deleted element
    b.disabled = True
Graph.wf.undo = undo
Graph.w.Edit.undo.on_click( Graph.wf.undo )

## Color

In [485]:
Graph.w.Edit.color = ipw.ColorPicker( )
#Graph.w.Edit.color

In [486]:
import time

In [487]:
Graph.w.Edit.color.time = 0
def color(b):
    if time.time() - Graph.w.Edit.color.time > .05:  #<-- make function run only ever 1/10 a second
        Graph.w.Edit.color.time = time.time()
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isNode() ){
                ele.style( "background-color", '"""+Graph.w.Edit.color.value+"""' )
            } else{
                ele.style( "line-color", '"""+Graph.w.Edit.color.value+"""' )
            }
        }
        """)          
Graph.wf.color = color
Graph.w.Edit.color.observe( Graph.wf.color )

## Background Color

In [488]:
Graph.w.Edit.background = ipw.Button( button_style='danger', icon='image', layout= ipw.Layout(width='auto') )
#Graph.w.Edit.background

In [489]:
def background(b):
    JS(f"document.getElementById('cy').style.backgroundColor = '{Graph.w.Edit.color.value}'")  
Graph.wf.background = background
Graph.w.Edit.background.on_click( Graph.wf.background )

## Size

In [457]:
Graph.w.Edit.size = ipw.IntSlider(value=30, min=1, max=200, continuous_update=True )
#Graph.w.Edit.size 

In [442]:
Graph.w.Edit.size.time = 0
def size(b):
    if time.time() - Graph.w.Edit.size.time > .05:  #<-- make function run only ever 1/10 a second
        Graph.w.Edit.size.time = time.time()
        JS("""
        for( i=0; i < cy.io.selection.length; i++ ){
            var ele = cy.io.selection[i]
            if( ele.isNode() ){
                ele.style( "width", """+str(Graph.w.Edit.size.value)+""" )
                ele.style( "height", """+str(Graph.w.Edit.size.value)+""" )
            } else{
                ele.style( "width", """+str(Graph.w.Edit.size.value/6)+""" )
            }
        }
        """)           
Graph.wf.size = size
Graph.w.Edit.size.observe( Graph.wf.size )

## ID Text

In [330]:
Graph.w.Edit.id = widgets.Text( description='ID :', continous_update = False  )#, continous_update = False )
Graph.w.Edit.id

Text(value='', description='ID :')

In [322]:
def setId(b):
    JS("""
    for( i=0; i < cy.io.selection.length; i++ ){
        var ele = cy.io.selection[i]
        if( ele.isNode() ){
            ele.style( "width", """+str(Graph.w.Edit.size.value)+""" )
            ele.style( "height", """+str(Graph.w.Edit.size.value)+""" )
        } else{
            ele.style( "width", """+str(Graph.w.Edit.size.value/6)+""" )
        }
    }
    """)           
Graph.wf.setId = setId
Graph.w.Edit.id.observe( Graph.wf.setId )

## Label Text

In [323]:
Graph.w.Edit.label = widgets.Text( description='Label :', continous_update = False  )
Graph.w.Edit.label

Text(value='', description='Label :')

## Search

In [305]:
Graph.w.Edit.search = widgets.Text( description='Search :', continuous_update=False )
#Graph.w.Edit.search

In [314]:
def search(b):
    JS(f"""
    cy.$(':selected').unselect()
    cy.$('{Graph.w.Edit.search.value}').select()
    """)   #<-- delete elements
Graph.wf.search = search
Graph.w.Edit.search.observe( Graph.wf.search )

## Node Shape

In [ ]:
Graph.w.Edit.shape = widgets.Dropdown( description='Shape :', continuous_update=False )
#Graph.w.Edit.search

In [ ]:
widgets.Dropdown(
    options=['circle', '2', '3'],
    value='2',
    description='Number:',
    disabled=False,
)

### Search Select

In [269]:
Graph.w.Edit.searchSelect = ipw.ToggleButton( icon='eye', layout= ipw.Layout(width='auto')  )
#Graph.w.Edit.searchSelect

In [270]:
def searchSelect(b):
    
    if Graph.w.Edit.searchSelect.value:   #<-- when activated
        Graph.w.Edit.searchSelect.icon = 'eye-slash'
        JS("""
        cy.io.to_hide = cy.elements().difference( cy.io.selection )
        cy.io.to_hide.style("visibility", "hidden")
        """)
    
    else: #<-- when not clicked
        Graph.w.Edit.searchSelect.icon = 'eye'
        JS("""cy.io.to_hide.style("visibility", "visible")""")
        
Graph.wf.searchSelect = searchSelect
Graph.w.Edit.searchSelect.observe( Graph.wf.searchSelect )



# Display Boxes

In [339]:
Graph.box = type("box",(),{})()

**Padding : ** \[ top/ right/ bottom/ left \]

## Style Box

In [518]:
Graph.box.color = ipw.HBox([ ipw.Label('Color :'), Graph.w.Edit.color, 
                            ipw.Label('    '), Graph.w.Edit.background ])

In [519]:
Graph.box.size = ipw.HBox([ipw.Label('Size :'), Graph.w.Edit.size ])

In [520]:
Graph.box.style = ipw.VBox([  Graph.box.size,
                              Graph.box.color])

In [525]:
#Graph.box.style

## Label Box

In [526]:
Graph.box.label = ipw.HTML("<h4>Not implemented yet</h4>")

## Variables Box

In [527]:
Graph.box.vars = ipw.HTML("<h4>Not implemented yet</h4>")

## Combine Style, Label, Variables

In [528]:
Graph.box.combo = widgets.Accordion(children=[ Graph.box.style ,Graph.box.label, Graph.box.vars])
Graph.box.combo.set_title(0, 'Style')
Graph.box.combo.set_title(1, 'Label')
Graph.box.combo.set_title(2, 'Variables')
#Graph.box.combo

## End Box

In [529]:
Graph.box.search = ipw.HBox([ Graph.w.Edit.search, Graph.w.Edit.searchSelect ])
Graph.box.search.layout.padding = '10px 0px 0px 0px'
#Graph.box.search

In [530]:
Graph.box.delete = ipw.HBox([ Graph.w.Edit.delete, Graph.w.Edit.undo ] )
Graph.box.delete.layout.justify_content = "center"
Graph.box.delete.layout.padding = '10px 0px 0px 0px'
#Graph.box.delete

In [531]:
Graph.box.end = ipw.VBox([Graph.box.search, 
                          Graph.box.delete ])
#Graph.box.end

## Box of Everything

In [545]:
Graph.box.all = ipw.VBox([ Graph.box.combo,
                           Graph.box.end])
#Graph.box.all

## Show Box Based on Selection

In [544]:
#Graph.box.size.layout.visibility = "visible"

In [548]:
def show(text):
    return Graph.box.all
Graph.box.show = show

# Create Graph

## Helper: <font color="magenta">JupyterCell</font>

<p></p>
 <font color="magenta">JupyterCell</font> is a useful class that allows you to execute a specified *jupyter cell* from the command <font color="magenta">JupyterCell</font><font color="blue">.execute()</font>. This is used to change which <font color="green">widget</font> is displayed inside of the visualizer's **editor cell**.

The editor box is the thing next to the visualizer where a user can change the graph. Notice that the contents of this box changes depending on what the user clicks on the control panel. The Control Panel/ Editor Box functionality is actually a little tricky to create in jupyter and require some special things to be built.


In [38]:
class JupyterCell():
 
    def __init__(self, cell_text, js_var, text_before="# ", ):
        self.cell_text= cell_text
        self.text_before= text_before
        self.js_var = js_var
        self.search()
    
    def search(self):
        JS("""
        var cell_text= "";
        """+self.js_var+"""= 0;  
        while( ! cell_text.includes('"""+self.text_before+self.cell_text+"""')  ){
            cell_text = Jupyter.notebook.get_cell("""+self.js_var+""").get_text();  //.element["0"].innerText #<-- Old Way
            """+self.js_var+"""++
        }
        """+self.js_var+"""--
        """)
    
    def execute(self):  #<-- This gets the Cell by using the cell text. The cell text must be unique to that cell
        self.search()
        JS("""
        IPython.notebook.execute_cells(["""+self.js_var+"""])
        """)


## Helper: <font color="magenta">Output</font><font color="orange">.JS</font>

**<font color="magenta">OutputJS</font>** is a very useful <font color="magenta">python class</font>/<font color="orange">javascript function</font> that allows variable to go from: <font color="orange">javascript</font> $\rightarrow$ <font color="blue">python</font>.

**Python Code**
```python
JS("OutputJS( 'howdy ho!' )")
OutputJS.var
```
** Output**
```
'howdy ho!'
```


In [214]:
class Output():
    var = ""
    JS("""function sleep(ms){ return new Promise( resolve => setTimeout(resolve, ms)); };  """)
    JS("""
    async function OutputJS(text){ 
        Jupyter.notebook.kernel.execute( 'Output.var = "'+String(text)+'"' );  //<-- write text to python variable: Output.var 
        count = 1
        while( Jupyter.notebook.kernel_busy && count < 25  ){                  //<-- wait for variable to be written, then continue
            console.log( Jupyter.notebook.kernel_busy  )
            count += 1
            await sleep(100);
        }
        console.log("one more sleep")
        console.log( Jupyter.notebook.kernel_busy )
        await sleep(1000);
    };  
    """)
    
    def JS(text):
        JS("""OutputJS("""+text+""")""")
        time.sleep(.1)
        return Output.var
        


In [215]:
JS("""OutputJS(x)""")
time.sleep(1)
Output.var

''

In [216]:
Output.var

'35'

In [195]:
time.sleep(3)


##  create graph and editor

Must re-execute this when debugging code

In [566]:
Graph.w.Edit.boxAll = ipw.VBox([ ipw.HBox([ Graph.w.Edit.color, Graph.w.Edit.background ]),
                                 Graph.w.Edit.size,
                                 ipw.HBox([ Graph.w.Edit.search, Graph.w.Edit.searchSelect ]),
                                 ipw.HBox([ Graph.w.Edit.delete, Graph.w.Edit.undo ]),
                               ])


In [567]:
cy = Graph()


Graph.canvas_box = JupyterCell("My Canvas Box", "canvas_num")
Graph.canvas_box.execute()


In [569]:
Graph.editor_box = JupyterCell("My Edit Box", "edit_num")
#Graph.editor_box.widg = ipw.HTML("<h5> </h5>") #<-- widget initially empty
Graph.editor_box.execute()


In [571]:
cy.js.addNode()
cy.js.addEdge()
cy.js.editSelector()

In [ ]:
JS("""
//cy.removeListener('tap', handler);
//cy.removeListener( "click" ) 
//cy.removeListener( "tap" )
//cy.removeListener( "select" )
//cy.removeListener( "unselect" )
""")

In [331]:
def selectColor(self):

checkit = cy.io.selection
checkit.style("overlay-color","gray")
checkit.style("overlay-opacity",".3")
checkit.style("overlay-padding","8px")

IndentationError: expected an indented block (<ipython-input-331-0ca47ed7f79b>, line 3)

In [47]:
JS(f"document.getElementById('cy').style.backgroundColor = '{Graph.w.Edit.color.value}'")  

In [563]:
Graph.editor_box.widg = Graph.box.show('1 node')

In [ ]:
"""JS(
Jupyter.notebook.kernel.execute("Graph.editor_box.widg = Graph.w.Edit.boxAll")
IPython.notebook.execute_cells([edit_num])  //<-- edit_num was setup by the Graph.editor_box
)"""

# Visualize (Mock Setup)

In [570]:
# My Edit Box
display( Graph.editor_box.widg  )



AttributeError: 'JupyterCell' object has no attribute 'widg'

In [568]:
# My Canvas Box
cy.canvasHtml()
cy.js.start()

In [ ]:
checkit = cy.elements()
var not_selected = cy.elements().difference( cy.io.selection )



In [426]:
ipw.Button(
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    icon='image',
    layout= ipw.Layout(width='auto')
)

Button(button_style='danger', icon='image', layout=Layout(width='auto'), style=ButtonStyle())

In [400]:
JS(f"cy.$('{Graph.w.Edit.search.value}').select()")  

In [46]:
f"cy.$('{Graph.w.Edit.search.value}').select()"


"cy.$('').select()"

In [248]:
JS("""
    cy.removeListener('select');
""")

In [238]:
JS("""
checkit = cy.io.selection
checkit.style("overlay-color","gray")
checkit.style("overlay-opacity",".3")
checkit.style("overlay-padding","8px")
""")

** NOw working on node/edge size**

In [53]:
JS("""
console.log( cy.io.selection[0].width() )
""")   

In [ ]:
cy.$(':selected')

checkit[1].isNode()
checkit[0].isEdge()



Old test function

In [ ]:
JS("""
    var check = 0;
    var check_var;

    cy.io.test = function(event){
        cy.io.selection = cy.$(':selected')
        
        check += 1
        console.log( String(check) +": selection changed")
        check_var = 
        cehck_var2 = event
    };

    cy.on('select unselect', 'node, edge', cy.io.test );
""")


In [ ]:
#import inspect
#lines = inspect.getsource(cy.js.addEdge)
#print(lines)

# Scratch Paper

In [ ]:
"""
JS(

cyto.layout({ name: 'circle'}).run();

)
"""

In [ ]:
"""cy.layout({ name: 'circle'}).run();"""